In [1]:
# ⌨️ add path:
data_path = "../data/Tutorial_3/ribs"

In [2]:
# check if data_path exists:
import os

if not os.path.exists(data_path):
    print("Please update your data path to an existing folder.")
elif not set(["train", "val", "test"]).issubset(set(os.listdir(data_path))):
    print("Please update your data path to the correct folder (should contain train, val and test folders).")
else:
    print("Congrats! You selected the correct folder :)")

Please update your data path to an existing folder.


In [3]:
# Install additional packages required for this tutorial
!pip install scikit-image
!pip install wandb
!pip install monai

In [4]:
import wandb
wandb.login()

wandb: Currently logged in as: jelmerwolterink. Use `wandb login --relogin` to force relogin


True

In [5]:
run = wandb.init(project='Example project', name='Example run', config={'dataset': 'VinDr-RibCXR'})

wandb: Tracking run with wandb version 0.16.6


wandb: Run data is saved locally in /Users/jmwolterink/Library/CloudStorage/OneDrive-UniversityofTwente/Teaching/DLMIA_2023_2024/dlmia-2024/notebooks/Tutorial3_Segmentation/wandb/run-20240424_151937-v7vx8g0y
wandb: Run `wandb offline` to turn off syncing.


wandb: Syncing run Example run


wandb: ⭐️ View project at https://wandb.ai/jelmerwolterink/Example%20project


wandb: 🚀 View run at https://wandb.ai/jelmerwolterink/Example%20project/runs/v7vx8g0y


In [6]:
import random
import time

# time.sleep(1) makes the script wait for 1 second, so the full script takes about 60 seconds to finish
for step in range(60):
    wandb.log({'Loss': random.random(), 'Accuracy': random.random()})
    time.sleep(1)
    
run.finish()

wandb:                                                                                


wandb: 
wandb: Run history:
wandb: Accuracy ▂▇██▂▁▆▅▄▄█▅█▆▇▆▂▄▄▅▄▆▆▂▄▂▃▆▅▆▂▄▄▂▅▄█▂█▂
wandb:     Loss ▅▁▂▅▅▄▄▂▂▄▆▄▆▇▆▆▅▇▇▃▄▅▇▃▁▅▄█▁█▆▂▂▆▆▃▂▁▆▆
wandb: 
wandb: Run summary:
wandb: Accuracy 0.23118
wandb:     Loss 0.68557
wandb: 


wandb: 🚀 View run Example run at: https://wandb.ai/jelmerwolterink/Example%20project/runs/v7vx8g0y
wandb: ⭐️ View project at: https://wandb.ai/jelmerwolterink/Example%20project
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)


wandb: Find logs at: ./wandb/run-20240424_151937-v7vx8g0y/logs


In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import monai
from PIL import Image
import torch

def build_dict_ribs(data_path, mode='train'):
    """
    This function returns a list of dictionaries, each dictionary containing the keys 'img' and 'mask' 
    that returns the path to the corresponding image.
    
    Args:
        data_path (str): path to the root folder of the data set.
        mode (str): subset used. Must correspond to 'train', 'val' or 'test'.
        
    Returns:
        (List[Dict[str, str]]) list of the dictionaries containing the paths of X-ray images and masks.
    """
    # test if mode is correct
    if mode not in ["train", "val", "test"]:
        raise ValueError(f"Please choose a mode in ['train', 'val', 'test']. Current mode is {mode}.")
    
    # define empty dictionary
    dicts = []
    # list all .png files in directory, including the path
    paths_xray = glob.glob(os.path.join(data_path, mode, 'img', '*.png'))
    # make a corresponding list for all the mask files
    for xray_path in paths_xray:
        if mode == 'test':
            suffix = 'val'
        else:
            suffix = mode
        # find the binary mask that belongs to the original image, based on indexing in the filename
        image_index = os.path.split(xray_path)[1].split('_')[-1].split('.')[0]
        # define path to mask file based on this index and add to list of mask paths
        mask_path = os.path.join(data_path, mode, 'mask', f'VinDr_RibCXR_{suffix}_{image_index}.png')
        if os.path.exists(mask_path):
            dicts.append({'img': xray_path, 'mask': mask_path})
    return dicts

class LoadRibData(monai.transforms.Transform):
    """
    This custom Monai transform loads the data from the rib segmentation dataset.
    Defining a custom transform is simple; just overwrite the __init__ function and __call__ function.
    """
    def __init__(self, keys=None):
        pass

    def __call__(self, sample):
        image = Image.open(sample['img']).convert('L') # import as grayscale image
        image = np.array(image, dtype=np.uint8)
        mask = Image.open(sample['mask']).convert('L') # import as grayscale image
        mask = np.array(mask, dtype=np.uint8)
        # mask has value 255 on rib pixels. Convert to binary array
        mask[np.where(mask==255)] = 1
        return {'img': image, 'mask': mask, 'img_meta_dict': {'affine': np.eye(2)}, 
                'mask_meta_dict': {'affine': np.eye(2)}}

In [8]:
# construct list of dictionaries
train_dict_list = build_dict_ribs(data_path, mode='train')
# construct CacheDataset from list of paths + transform
train_dataset = monai.data.CacheDataset(train_dict_list, transform=LoadRibData())

In [9]:
val_dict_list = build_dict_ribs(data_path, mode='val')
test_dict_list = build_dict_ribs(data_path, mode='test')
val_dataset = monai.data.CacheDataset(val_dict_list, transform=LoadRibData())
test_dataset = monai.data.CacheDataset(test_dict_list, transform=LoadRibData())

print(f'{train_dataset.__len__()=}')
print(f'{val_dataset.__len__()=}')
print(f'{test_dataset.__len__()=}')

SyntaxError: invalid syntax (<fstring>, line 1)

In [10]:
for i in range(5):
    sample = train_dataset[i]
    img = sample['img']
    mask = sample['mask']
    print(f'{img.shape=}, {mask.shape=}')

SyntaxError: invalid syntax (<fstring>, line 1)

In [11]:
bins = np.linspace(0, 255, 50)
for i in range(5):
    sample = train_dataset[i]
    img = sample['img']
    img = img.flatten()
    plt.hist(img, bins, alpha=0.5, label=i)
plt.show()

ZeroDivisionError: integer division or modulo by zero

In [12]:
def visualize_rib_sample(sample, title=None):
    # Visualize the x-ray and overlay the mask, using the dictionary as input
    image = np.squeeze(sample['img'])
    mask = np.squeeze(sample['mask'])
    plt.figure(figsize=[10,7])
    plt.imshow(image, 'gray')
    overlay_mask = np.ma.masked_where(mask == 0, mask == 1)
    plt.imshow(overlay_mask, 'Greens', alpha = 0.7, clim=[0,1], interpolation='nearest')
    if title is not None:
        plt.title(title)
    plt.show()

In [13]:
sample_dict = train_dataset[0]
visualize_rib_sample(sample_dict)

ZeroDivisionError: integer division or modulo by zero

In [14]:
# Load sample
index = np.random.choice(np.arange(len(train_dataset))) # This picks a random sample, but you can change this value
sample_dict = train_dataset[index]
visualize_rib_sample(sample_dict, title="Original sample")

# Add channels
add_channels_transform = monai.transforms.AddChanneld(keys=['img', 'mask']) # Initialize the transform
channels_sample_dict = add_channels_transform(sample_dict) # Apply the transform
print("Size of the image before AddChanneld transform", sample_dict["img"].shape)
print("Size of the image after AddChanneld transform", channels_sample_dict["img"].shape)

# Random flip
# here we define the keys, the probability that the flip is performed and the axis to flip over
random_flip_transform = monai.transforms.RandFlipd(keys=['img', 'mask'], prob=1, spatial_axis=1)
# We put a probability of 1 to always flip the image for visualization purposes.
# Please DO NOT DO THAT in the rest of the notebook.
flipped_sample_dict = random_flip_transform(channels_sample_dict)
visualize_rib_sample(flipped_sample_dict, title="(Not quite randomly) flipped sample")

# Random rotation
# Here we define the keys in the dictionary that contain the data, the rotation range, but also the interpolation mode. 
# The interpolation mode defines how new pixel values for the rotated image are computed. 
# Note that these differ between mask and image, as we want to keep binary labels for the masks, and (bi)linear interpolation
# would result in scalar values between 0 and 1.
random_rotation_transform = monai.transforms.RandRotated(keys=['img', 'mask'], range_x=np.pi/4, prob=1, mode=['bilinear', 'nearest'])
rotated_sample_dict = random_rotation_transform(channels_sample_dict)
visualize_rib_sample(rotated_sample_dict, title="Randomly rotated sample")

ValueError: 'a' cannot be empty unless no samples are taken

In [15]:
sample_dict = train_dataset[0]

# Create the composed transform
add_channels_transform = monai.transforms.AddChanneld(keys=['img', 'mask']) # Initialize the transform
random_flip_transform = monai.transforms.RandFlipd(keys=['img', 'mask'], prob=1, spatial_axis=1)
random_rotation_transform = monai.transforms.RandRotated(keys=['img', 'mask'], range_x=np.pi/4, prob=1, mode=['bilinear', 'nearest'])

transforms = monai.transforms.Compose([
    add_channels_transform,
    random_flip_transform,
    random_rotation_transform
])

# Apply this new single transform to sample_dict
transformed_sample = transforms(sample_dict)
visualize_rib_sample(transformed_sample, title="Transformed sample")

ZeroDivisionError: integer division or modulo by zero

In [16]:
train_transform = monai.transforms.Compose([
    LoadRibData(),
    monai.transforms.AddChanneld(keys=['img', 'mask']),
    monai.transforms.ScaleIntensityd(keys=['img'],minv=0, maxv=1),
    monai.transforms.Zoomd(keys=['img', 'mask'], zoom=0.25, keep_size=False, mode=['bilinear', 'nearest']),
    monai.transforms.RandFlipd(keys=['img', 'mask'], prob=0.5, spatial_axis=1),
    monai.transforms.RandSpatialCropd(keys=['img', 'mask'], roi_size=[256,256], random_size=False)
])

train_dataset = monai.data.CacheDataset(train_dict_list, transform=train_transform)

for i in range(5):
    visualize_rib_sample(train_dataset[i], title=f"Transformed sample {i}")

/Users/jmwolterink/opt/anaconda3/envs/notebook_collaboration/lib/python3.7/site-packages/monai/utils/deprecate_utils.py:107: FutureWarning: <class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.
  warn_deprecated(obj, msg, warning_category)


ZeroDivisionError: integer division or modulo by zero

In [17]:
def from_compose_to_list(transform_compose):
    """
    Transform an object monai.transforms.Compose in a list fully describing the transform.
    /!\ Random seed is not saved, then reproducibility is not enabled.
    """
    from copy import deepcopy
        
    if not isinstance(transform_compose, monai.transforms.Compose):
        raise TypeError("transform_compose should be a monai.transforms.Compose object.")
    
    output_list = list()
    for transform in transform_compose.transforms:
        kwargs = deepcopy(vars(transform))
        
        # Remove attributes which are not arguments
        args = list(transform.__init__.__code__.co_varnames[1: transform.__init__.__code__.co_argcount])
        for key, obj in vars(transform).items():
            if key not in args:
                del kwargs[key]

        output_list.append({"class": transform.__class__, "kwargs": kwargs})
    return output_list

def from_list_to_compose(transform_list):
    """
    Transform a list in the corresponding monai.transforms.Compose object.
    """
    
    if not isinstance(transform_list, list):
        raise TypeError("transform_list should be a list.")
    
    pre_compose_list = list()
    
    for transform_dict in transform_list:
        if not isinstance(transform_dict, dict) or 'class' not in transform_dict or 'kwargs' not in transform_dict:
            raise TypeError("transform_list should only contains dicts with keys ['class', 'kwargs']")
        
        try:
            transform = transform_dict['class'](**transform_dict['kwargs'])
        except TypeError: # Classes have been converted to str after saving
            transform = eval(transform_dict['class'].replace("__main__.", ""))(**transform_dict['kwargs'])
            
        pre_compose_list.append(transform)
        
    return monai.transforms.Compose(pre_compose_list)

In [18]:
train_loader = monai.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
print(next(iter(train_loader))['img'].shape)

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [19]:
validation_dict = build_dict_ribs(data_path, mode='val')

validation_transforms = monai.transforms.Compose([
    LoadRibData(),
    monai.transforms.AddChanneld(keys=['img', 'mask']),
    monai.transforms.ScaleIntensityd(keys=['img'],minv=0, maxv=1),
    monai.transforms.Resized(keys=['img', 'mask'], spatial_size=[256,256])
])
validation_data = monai.data.CacheDataset(validation_dict, transform=validation_transforms)
validation_loader = monai.data.DataLoader(validation_data, batch_size=16)
print(next(iter(validation_loader))['img'].shape)

StopIteration: 

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'The used device is {device}')

The used device is cpu


In [21]:
model = monai.networks.nets.UNet(
    spatial_dims=2,
    in_channels=1,
    out_channels=1,
    channels=(8, 16, 32, 64, 128),
    strides=(2, 2, 2, 2),
    num_res_units=2,
).to(device)

In [22]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'U-Net has 5 layers (see "channels" parameter) and {num_params} parameters')

U-Net has 5 layers (see "channels" parameter) and 406804 parameters


In [23]:
loss_function =  monai.losses.DiceLoss(sigmoid=True, batch=True)

In [24]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [25]:
from tqdm import tqdm
import wandb

run = wandb.init(
    project='tutorial3_segmentation',
    name='test',
    config={
        'loss function': str(loss_function), 
        'lr': optimizer.param_groups[0]["lr"],
        'transform': from_compose_to_list(train_transform),
        'batch_size': train_loader.batch_size,
    }
)
# Do not hesitate to enrich this list of settings to be able to correctly keep track of your experiments!
# For example you should add information on your model...

run_id = run.id # We remember here the run ID to be able to write the evaluation metrics

def wandb_masks(mask_output, mask_gt):
    """ Function that generates a mask dictionary in format that W&B requires """

    # Apply sigmoid to model ouput and round to nearest integer (0 or 1)
    sigmoid = torch.nn.Sigmoid()
    mask_output = sigmoid(mask_output)
    mask_output = torch.round(mask_output)

    # Transform masks to numpy arrays on CPU
    # Note: .squeeze() removes all dimensions with a size of 1 (here, it makes the tensors 2-dimensional)
    # Note: .detach() removes a tensor from the computational graph to prevent gradient computation for it
    mask_output = mask_output.squeeze().detach().cpu().numpy()
    mask_gt = mask_gt.squeeze().detach().cpu().numpy()

    # Create mask dictionary with class label and insert masks
    class_labels = {1: 'ribs'}
    masks = {
        'predictions': {'mask_data': mask_output, 'class_labels': class_labels},
        'ground truth': {'mask_data': mask_gt, 'class_labels': class_labels}
    }
    return masks

def log_to_wandb(epoch, train_loss, val_loss, batch_data, outputs):
    """ Function that logs ongoing training variables to W&B """

    # Create list of images that have segmentation masks for model output and ground truth
    log_imgs = [wandb.Image(img, masks=wandb_masks(mask_output, mask_gt)) for img, mask_output,
                mask_gt in zip(batch_data['img'], outputs, batch_data['mask'])]

    # Send epoch, losses and images to W&B
    wandb.log({'epoch': epoch, 'train_loss': train_loss, 'val_loss': val_loss, 'results': log_imgs})
    
for epoch in tqdm(range(200)):
    
    # training
    model.train()    
    epoch_loss = 0
    step = 0
    for batch_data in train_loader: 
        step += 1
        optimizer.zero_grad()
        outputs = model(batch_data["img"].float().to(device))
        loss = loss_function(outputs, batch_data["mask"].to(device))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    train_loss = epoch_loss/step
    
    # validation
    step = 0
    val_loss = 0
    for batch_data in validation_loader:
        step += 1
        model.eval()
        outputs = model(batch_data['img'].float().to(device))
        loss = loss_function(outputs, batch_data['mask'].to(device))
        val_loss+= loss.item()
    val_loss = val_loss / step
    log_to_wandb(epoch, train_loss, val_loss, batch_data, outputs)

# Store the network parameters        
torch.save(model.state_dict(), r'trainedUNet.pt')
run.finish()

NameError: name 'train_loader' is not defined

In [26]:
def visual_evaluation(sample, model):
    """
    Allow the visual inspection of one sample by plotting the X-ray image, the ground truth (green)
    and the segmentation map produced by the network (red).
    
    Args:
        sample (Dict[str, torch.Tensor]): sample composed of an X-ray ('img') and a mask ('mask').
        model (torch.nn.Module): trained model to evaluate.
    """
    model.eval()
    inferer = monai.inferers.SlidingWindowInferer(roi_size=[256, 256])
    discrete_transform = monai.transforms.AsDiscrete(logit_thresh=0.5, threshold_values=True)
    Sigmoid = torch.nn.Sigmoid()
    with torch.no_grad():
        output = discrete_transform(Sigmoid(inferer(sample['img'].to(device), network=model).cpu())).squeeze()
    
    fig, ax = plt.subplots(1,3, figsize = [12, 10])
    # Plot X-ray image
    ax[0].imshow(sample["img"].squeeze(), 'gray')    
    ax[1].imshow(sample["img"].squeeze(), 'gray')
    # Plot ground truth
    mask = np.squeeze(sample['mask'])
    overlay_mask = np.ma.masked_where(mask == 0, mask == 1)
    ax[1].imshow(overlay_mask, 'Greens', alpha = 0.7, clim=[0,1], interpolation='nearest')
    ax[1].set_title('Ground truth')
    # Plot output
    overlay_output = np.ma.masked_where(output < 0.1, output >0.99)
    ax[2].imshow(sample['img'].squeeze(), 'gray')
    ax[2].imshow(overlay_output, 'Reds', alpha = 0.7, clim=[0,1])
    ax[2].set_title('Prediction')
    plt.show()

In [27]:
test_dict = build_dict_ribs(data_path, mode='test')
test_transform = monai.transforms.Compose([
        LoadRibData(),
        monai.transforms.AddChanneld(keys=['img', 'mask']),
        monai.transforms.ScaleIntensityd(keys=['img'],minv=0, maxv=1),
        monai.transforms.Zoomd(keys=['img', 'mask'], zoom=0.25, keep_size=False, mode=['bilinear', 'nearest']),
    ]
)
test_set = monai.data.CacheDataset(test_dict, transform=test_transform)
test_loader = monai.data.DataLoader(test_set, batch_size=1)

for sample in test_loader:
    visual_evaluation(sample, model)

In [28]:
def compute_metric(dataloader, model, metric_fn):
    """
    This function computes the average value of a metric for a data set.
    
    Args:
        dataloader (monai.data.DataLoader): dataloader wrapping the dataset to evaluate.
        model (torch.nn.Module): trained model to evaluate.
        metric_fn (function): function computing the metric value from two tensors:
            - a batch of outputs,
            - the corresponding batch of ground truth masks.
        
    Returns:
        (float) the mean value of the metric
    """
    model.eval()
    inferer = monai.inferers.SlidingWindowInferer(roi_size=[256, 256])
    discrete_transform = monai.transforms.AsDiscrete(threshold=0.5)
    Sigmoid = torch.nn.Sigmoid()
    
    mean_value = 0
    
    for sample in dataloader:
        with torch.no_grad():
            output = discrete_transform(Sigmoid(inferer(sample['img'].to(device), network=model).cpu()))
        mean_value += metric_fn(output, sample["mask"])
    
    return (mean_value / len(dataloader)).item()

In [29]:
api = wandb.Api()
run = api.run(f"tutorial3_segmentation/{run_id}")

NameError: name 'run_id' is not defined

In [30]:
metric_fn = monai.metrics.compute_meandice
dice = compute_metric(test_loader, model, metric_fn)
run.summary["dice"] = dice
print(f"Dice on test set: {dice:.3f}")

ZeroDivisionError: division by zero

In [31]:
metric_fn = monai.metrics.compute_hausdorff_distance
Hausdorff_dist = compute_metric(test_loader, model, metric_fn)
run.summary["Hausdorff_dist"] = Hausdorff_dist
print(f"Hausdorff distance on test set: {Hausdorff_dist:.3f}")

ZeroDivisionError: division by zero

In [32]:
from skimage import measure

def make_dummy_sample(sample):
    M = sample['mask'].squeeze()
    labels = measure.label(M)
    dummy_labels = np.zeros((labels.shape[0], labels.shape[1]))
    for i in np.unique(labels):
        if i > 0:
            mask_locs = np.where(labels == i)
            limits = [np.min(mask_locs[0]), np.max(mask_locs[0]), np.min(mask_locs[1]), np.max(mask_locs[1])]
            dummy_labels[limits[0]:limits[1], limits[2]:limits[3]] = 1
    return torch.tensor(dummy_labels)

In [33]:
sample = test_set[0]
visualize_rib_sample(sample, title = 'original mask')
visualize_rib_sample({'img': sample['img'], 'mask': make_dummy_sample(sample)}, title = 'dummy segmentation masks')

ZeroDivisionError: integer division or modulo by zero

In [34]:
def compute_metric_dummy(dataloader, metric_fn):
    """
    This function computes the average value of a metric for a data set using the dummy segmentation masks
    
    Args:
        dataloader (monai.data.DataLoader): dataloader wrapping the dataset to evaluate.
        metric_fn (function): function computing the metric value from two tensors:
            - a batch of outputs,
            - the corresponding batch of ground truth masks.
        
    Returns:
        (float) the mean value of the metric
    """
    
    mean_value = 0
    A = monai.transforms.AddChannel()
    
    for sample in dataloader:
        output = make_dummy_sample(sample)
        mean_value += metric_fn(A(A(output)), sample["mask"])
    
    return (mean_value / len(dataloader)).item()

In [35]:
print(f'Mean Dice: {compute_metric_dummy(test_loader, monai.metrics.compute_meandice):.3f}')

ZeroDivisionError: division by zero

In [36]:
# ⌨️ code your answer here